In [1]:
import requests

In [1]:
import requests

offset_val = 0

# Define the base URL without filter parameters
base_url = "https://data.exactspace.co/exactapi/units/60ae9143e284d016d3559dfb/activities"

all_objects = []  # List to store all fetched objects

while len(all_objects) < 900:  # Continue fetching until desired number is reached
    
    # Construct the URL with the updated offset value
    url = f"{base_url}?filter=%7B%20%20%20%20%20%22where%22%3A%20%7B%20%20%20%20%20%20%20%20%20%22type%22%3A%20%22task%22%20%20%7D%2C%20%20%20%20%20%22order%22%3A%20%22createdOn%20DESC%22%2C%20%20%20%20%20%22limit%22%3A%2010%2C%20%22offset%22%3A{offset_val}%20%7D&access_token=Ziv35SH8DYvWbh0ZL8HMhOCESfQTwajS7j8iHfb1A3pEoSO6P8NraCLmCD1g7iAH"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        objects_fetched = len(data)  # Number of objects fetched in this request
        all_objects.extend(data)      # Add fetched objects to the list
        offset_val += 10
        print("offset success:", offset_val)
    else:
        print("Failed to fetch data. Status code:", response.status_code)
        break

print("Total objects fetched:", len(all_objects))


offset success: 10
offset success: 20
offset success: 30
offset success: 40
offset success: 50
offset success: 60
offset success: 70
offset success: 80
offset success: 90
offset success: 100
offset success: 110
offset success: 120
offset success: 130
offset success: 140
offset success: 150
offset success: 160
offset success: 170
offset success: 180
offset success: 190
offset success: 200
offset success: 210
offset success: 220
offset success: 230
offset success: 240
offset success: 250
offset success: 260
offset success: 270
offset success: 280
offset success: 290
offset success: 300
offset success: 310
offset success: 320
offset success: 330
offset success: 340
offset success: 350
offset success: 360
offset success: 370
offset success: 380
offset success: 390
offset success: 400
offset success: 410
offset success: 420
offset success: 430
offset success: 440
offset success: 450
offset success: 460
offset success: 470
offset success: 480
offset success: 490
offset success: 500
offset su

In [23]:
all_objects[6]

{'type': 'task',
 'voteAcceptCount': 0,
 'voteRejectCount': 0,
 'acceptedUserList': [],
 'rejectedUserList': [],
 'dueDate': '2024-04-23T06:38:57.962Z',
 'assignee': None,
 'source': 'PM',
 'team': 'Maintenance',
 'createdBy': None,
 'createdOn': '2024-04-22T06:38:40.000Z',
 'siteId': '65817c061be4c00007dc7f02',
 'subTaskOf': None,
 'subTasks': [],
 'chats': [],
 'chatOf': None,
 'taskPriority': None,
 'updateHistory': [{'action': 'Pulse created this task',
   'on': '2024-04-22T06:38:40.000Z',
   'by': ''}],
 'id': '6625b8aa3f0e4100065ba5f9',
 'unitsId': '65817c16dcd5de0007bbbf5d',
 'status': 'new',
 'sourceURL': '/rotary-assets/equipment/658192b61be4c00007dc8136?unitId=65817c16dcd5de0007bbbf5d&pm=660fbf49e009d9000790f00b',
 'pmId': '660fbf49e009d9000790f00b',
 'taskGeneratedBy': 'system',
 'recur': True,
 'content': [{'type': 'title', 'value': 'Test 2'}]}

In [60]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # Write tables
    start_row += 1  # Adding some empty rows between subtitles and tables
    for table_data in tables:
        # Set column width
        worksheet.set_column('A:F', 33)
        # Bold and light background color for the first row
        bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
        for j, value in enumerate(table_data[0]):
            worksheet.write(start_row, j, value, bold_format)
        start_row += 1
        for row_data in table_data[1:]:
            for j, value in enumerate(row_data):
                worksheet.write(start_row, j, value)
            start_row += 1
        start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionary
    status_count = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}

    # Count statuses
    for obj in all_objects:
        status = obj.get("status", "--")
        status_count[status] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})

    # Write headers
    statistics_sheet.write('A1', 'Status', bold_format)
    statistics_sheet.write('B1', 'Count', bold_format)

    # Write status counts
    for idx, (status, count) in enumerate(status_count.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj["content"])
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [61]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # Write tables
    start_row += 1  # Adding some empty rows between subtitles and tables
    for table_data in tables:
        # Set column width
        worksheet.set_column('A:F', 33)
        # Bold and light background color for the first row
        bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
        for j, value in enumerate(table_data[0]):
            worksheet.write(start_row, j, value, bold_format)
        start_row += 1
        for row_data in table_data[1:]:
            for j, value in enumerate(row_data):
                worksheet.write(start_row, j, value)
            start_row += 1
        start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}

    # Count statuses
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})

    # Write headers
    statistics_sheet.write('A1', 'Status', bold_format)
    statistics_sheet.write('B1', 'Count (Recommendation)', bold_format)
    statistics_sheet.write('C1', 'Count (Shift Report)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=2):
        statistics_sheet.write(f'C{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [83]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # Write tables
    start_row += 1  # Adding some empty rows between subtitles and tables
    for table_data in tables:
        # Set column width
        worksheet.set_column('A:F', 33)
        # Bold and light background color for the first row
        bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
        for j, value in enumerate(table_data[0]):
            worksheet.write(start_row, j, value, bold_format)
        start_row += 1
        for row_data in table_data[1:]:
            for j, value in enumerate(row_data):
                worksheet.write(start_row, j, value)
            start_row += 1
        start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})
    statistics_sheet.set_column('A:Z', 25)

    # Write headers for Recommendation
    statistics_sheet.write('A1', 'Status(Recommendations)', bold_format)
    statistics_sheet.write('B1', 'Count (Recommendations)', bold_format)
    statistics_sheet.write('C1', 'Action(Recommendations)', bold_format)
    statistics_sheet.write('D1', 'Count(Recommendations)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=2):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

    # Write headers for Shift Report
    statistics_sheet.write('F1', 'Status(Shift Report)', bold_format)
    statistics_sheet.write('G1', 'Count(Shift Report)', bold_format)
    statistics_sheet.write('H1', 'Action(Shift Report)', bold_format)
    statistics_sheet.write('I1', 'Count(Shift Report)', bold_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=2):
        statistics_sheet.write(f'F{idx}', status)
        statistics_sheet.write(f'G{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=2):
        statistics_sheet.write(f'H{idx}', count_type)
        statistics_sheet.write(f'I{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [71]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # If not update history, write tables
    if update_history:
        # Write tables
        start_row += 1  # Adding some empty rows between subtitles and tables
        for table_data in tables:
            # Set column width
            worksheet.set_column('A:F', 33)
            # Bold and light background color for the first row
            bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
            for j, value in enumerate(table_data[0]):
                worksheet.write(start_row, j, value, bold_format)
            start_row += 1
            for row_data in table_data[1:]:
                for j, value in enumerate(row_data):
                    worksheet.write(start_row, j, value)
                start_row += 1
            start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})
    statistics_sheet.set_column('A:I', 18)

    # Write headers for Recommendation
    statistics_sheet.write('A1', 'Status(Recommendations)', bold_format)
    statistics_sheet.write('B1', 'Count (Recommendations)', bold_format)
    statistics_sheet.write('C1', 'Action(Recommendations)', bold_format)
    statistics_sheet.write('D1', 'Count(Recommendations)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=2):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

    # Write headers for Shift Report
    statistics_sheet.write('F1', 'Status(Shift Report)', bold_format)
    statistics_sheet.write('G1', 'Count(Shift Report)', bold_format)
    statistics_sheet.write('H1', 'Action(Shift Report)', bold_format)
    statistics_sheet.write('I1', 'Count(Shift Report)', bold_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=2):
        statistics_sheet.write(f'F{idx}', status)
        statistics_sheet.write(f'G{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=2):
        statistics_sheet.write(f'H{idx}', count_type)
        statistics_sheet.write(f'I{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [76]:
import pandas as pd
from datetime import datetime

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # If not update history, write tables
    if update_history:
        # Write tables
        start_row += 1  # Adding some empty rows between subtitles and tables
        for table_data in tables:
            # Set column width
            worksheet.set_column('A:F', 33)
            # Bold and light background color for the first row
            bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
            for j, value in enumerate(table_data[0]):
                worksheet.write(start_row, j, value, bold_format)
            start_row += 1
            for row_data in table_data[1:]:
                for j, value in enumerate(row_data):
                    worksheet.write(start_row, j, value)
                start_row += 1
            start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Get the interval between the first and last object's createdOn times
    first_created_time = None
    last_created_time = None
    for obj in all_objects:
        created_time = datetime.strptime(obj.get("createdOn", ""), "%Y-%m-%dT%H:%M:%S.%fZ")
        if first_created_time is None or created_time < first_created_time:
            first_created_time = created_time
        if last_created_time is None or created_time > last_created_time:
            last_created_time = created_time
    
    interval = (last_created_time - first_created_time).days  # Interval in days

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})
    statistics_sheet.set_column('A:G', 25)

    # Write title and interval
    statistics_sheet.write('B1:E1', 'Statistics', bold_format)
    statistics_sheet.write('A3', 'First Created On', bold_format)
    statistics_sheet.write('B3', 'Last Created On', bold_format)
    statistics_sheet.write('C3', 'Interval (Days)', bold_format)

    statistics_sheet.write('A4', first_created_time.strftime("%Y-%m-%d %H:%M:%S"))
    statistics_sheet.write('B4', last_created_time.strftime("%Y-%m-%d %H:%M:%S"))
    statistics_sheet.write('C4', interval)

    # Write headers for Recommendation
    statistics_sheet.write('A7', 'Status(Recommendations)', bold_format)
    statistics_sheet.write('B7', 'Count (Recommendations)', bold_format)
    statistics_sheet.write('C7', 'Action(Recommendations)', bold_format)
    statistics_sheet.write('D7', 'Count(Recommendations)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=8):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=8):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

    # Write headers for Shift Report
    statistics_sheet.write('A15', 'Status(Shift Report)', bold_format)
    statistics_sheet.write('B15', 'Count(Shift Report)', bold_format)
    statistics_sheet.write('C15', 'Action(Shift Report)', bold_format)
    statistics_sheet.write('D15', 'Count(Shift Report)', bold_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=16):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=16):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [3]:
import pandas as pd
from datetime import datetime

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # If not update history, write tables
    if update_history:
        # Write tables
        start_row += 1  # Adding some empty rows between subtitles and tables
        for table_data in tables:
            # Set column width
            worksheet.set_column('A:F', 33)
            # Bold and light background color for the first row
            bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
            for j, value in enumerate(table_data[0]):
                worksheet.write(start_row, j, value, bold_format)
            start_row += 1
            for row_data in table_data[1:]:
                for j, value in enumerate(row_data):
                    worksheet.write(start_row, j, value)
                start_row += 1
            start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Get the interval between the first and last object's createdOn times
    first_created_time = None
    last_created_time = None
    for obj in all_objects:
        created_time = datetime.strptime(obj.get("createdOn", ""), "%Y-%m-%dT%H:%M:%S.%fZ")
        if first_created_time is None or created_time < first_created_time:
            first_created_time = created_time
        if last_created_time is None or created_time > last_created_time:
            last_created_time = created_time
    
    interval = (last_created_time - first_created_time).days  # Interval in days

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")
    statistics_sheet.set_column('A:Z', 18)

    # Define bold format and cell formatting
    bold_format = workbook.add_format({'bold': True})
    header_format = workbook.add_format({'bold': True, 'bg_color': '#0070C0', 'font_color': 'white', 'align': 'center', 'valign': 'vcenter'})
    data_format = workbook.add_format({'align': 'center', 'valign': 'vcenter'})
    subtitle_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})

    # Write title and interval
    statistics_sheet.merge_range('B1:E1', 'Statistics', header_format)
    statistics_sheet.merge_range('A3:A4', 'Created On', subtitle_format)
    statistics_sheet.merge_range('B3:B4', 'Last Created On', subtitle_format)
    statistics_sheet.merge_range('C3:C4', 'Interval (Days)', subtitle_format)

    statistics_sheet.write('A5', first_created_time.strftime("%Y-%m-%d %H:%M:%S"), data_format)
    statistics_sheet.write('B5', last_created_time.strftime("%Y-%m-%d %H:%M:%S"))
    statistics_sheet.write('C5', interval, data_format)

    # Write headers for Recommendation
    statistics_sheet.merge_range('A7:D7', 'Recommendations', header_format)
    statistics_sheet.write('A8', 'Status', header_format)
    statistics_sheet.write('B8', 'Count', header_format)
    statistics_sheet.write('C8', 'Action', header_format)
    statistics_sheet.write('D8', 'Count', header_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=9):
        statistics_sheet.write(f'A{idx}', status, data_format)
        statistics_sheet.write(f'B{idx}', count, data_format)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=9):
        statistics_sheet.write(f'C{idx}', count_type, data_format)
        statistics_sheet.write(f'D{idx}', count, data_format)

    # Write headers for Shift Report
    statistics_sheet.merge_range('F7:I7', 'Shift Report', header_format)
    statistics_sheet.write('F8', 'Status', header_format)
    statistics_sheet.write('G8', 'Count', header_format)
    statistics_sheet.write('H8', 'Action', header_format)
    statistics_sheet.write('I8', 'Count', header_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=9):
        statistics_sheet.write(f'F{idx}', status, data_format)
        statistics_sheet.write(f'G{idx}', count, data_format)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=9):
        statistics_sheet.write(f'H{idx}', count_type, data_format)
        statistics_sheet.write(f'I{idx}', count, data_format)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports_2204.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [8]:
len(all_objects)

900

In [5]:
import pandas as pd
import json
import requests
from datetime import datetime

tagList = ["GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric"]

def getValues4(tagList, startDate, endDate):
    url =  "https://data.exactspace.co/kairosapi/api/v1/datapoints/query"
    d = {
        "metrics": [
            {
                "tags": {},
                "name": "",
            }
        ],
        "plugins": [],
        "cache_time": 0,
        "start_absolute": startDate,
        "end_absolute": endDate
    }
    dfs = []
    for tag in tagList:
        d['metrics'][0]['name'] = tag
        res = requests.post(url=url, json=d)
        values = json.loads(res.content)
        df = pd.DataFrame(values["queries"][0]["results"][0]['values'], columns=['time', values["queries"][0]["results"][0]['name']])
        df['time'] = pd.to_datetime(df['time'], unit='ms') + pd.Timedelta(hours=5.5)
        df.sort_values(by='time', inplace=True)
        df = df.drop_duplicates(subset=["time", tag])
        if df.shape[0] < 10:
            pass
        else:
            dfs.append(df)
    df = dfs[0]
    for df_ in dfs[1:]:
        df = pd.merge(df, df_, on='time')
    return df

# Function to convert ISO 8601 timestamp to epoch timestamp
def iso_to_epoch(iso_timestamp):
    return int(datetime.fromisoformat(iso_timestamp[:-1]).timestamp())

# Extract recommendation data from JSON and create DataFrame
def extract_recommendations(all_objects):
    recommendations = []
    for obj in all_objects:
        title = obj.get("content", [{}])[0].get("value", "")
        if "GAP: GSD Recommendations" in title:
            recommendation_text = obj.get("content", [{}])[1].get("value", "")  # Assuming the recommendation text is in the second item of the content list
            update_history = obj.get("updateHistory", [])
            update_actions = [entry.get("action", "") for entry in update_history]  # Extract actions from updateHistory
            actions_str = "\n".join(update_actions)  # Concatenate actions into a single string separated by newline
            created_time = obj.get("createdOn")
            due_time = obj.get("dueDate")
            startDate = iso_to_epoch(created_time) * 1000
            endDate = iso_to_epoch(due_time) * 1000
            # Fetch data for the specified time range
            data = getValues4(tagList, startDate, endDate)
            data = data.tail(1)
            data = data.round(4)
            data = data.drop(columns=['time'])
            density = data.to_dict('records')
            result = density[0].get("GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric")
            if result > 1.65:
                density_achieved = "Yes"
            else:
                density_achieved = "No"
            recommendations.append({"ID": obj.get("id", ""), "Recommendation": recommendation_text, "Density Achieved": density_achieved, "Update Actions": actions_str})
    return recommendations

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    recommendations = extract_recommendations(all_objects)
    if not recommendations:
        print("No recommendations found.")
        return
    
    df = pd.DataFrame(recommendations)
    
    # Define writer
    writer = pd.ExcelWriter("recommendations.xlsx", engine="xlsxwriter")
    df.to_excel(writer, index=False, sheet_name="Recommendations")
    worksheet = writer.sheets["Recommendations"]
    
    # Set column width
    worksheet.set_column('A:Z', 15)  # Set column width for ID
    
    # Increase row height for cells with long text
    for i, text in enumerate(df["Recommendation"], start=1):
        num_lines = text.count("\n") + 1  # Count the number of lines in the text
        if num_lines > 1:
            worksheet.set_row(i, 20 * num_lines)  # Set row height to accommodate the text
        
    writer.save()
    print("Excel file saved successfully.")

# Assuming `all_objects` is defined somewhere
extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [6]:
import pandas as pd

# Load the Excel file
excel_file = 'recommendations.xlsx'
df = pd.read_excel(excel_file)

# Define the phrase to remove
phrase_to_remove = "This task is created by Pulse."

# Remove the phrase from the specified column
df['Update Actions'] = df['Update Actions'].str.replace(phrase_to_remove, '')

# Save the modified DataFrame back to Excel
output_file = 'final_report.xlsx'
df.to_excel(output_file, index=False)

print("Data cleaned and saved successfully.")


Data cleaned and saved successfully.


c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
